In [1]:
from langchain_google_genai import GoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

c:\Users\Administrator\crm-venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Connect to Gemini llm

In [2]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
llm("What is an AI assistant")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14428\3708976949.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  llm("What is an AI assistant")


'An AI assistant is a type of artificial intelligence (AI) software that can perform tasks or provide information to users. AI assistants are typically powered by machine learning and natural language processing (NLP) technologies, and they can be used for a wide range of tasks, including:\n\n* **Answering questions:** AI assistants can answer questions about a variety of topics, from the weather to the latest news.\n* **Providing information:** AI assistants can provide information on a variety of topics, such as the location of a restaurant or the definition of a word.\n* **Performing tasks:** AI assistants can perform tasks such as setting alarms, sending emails, and playing music.\n* **Helping with decision-making:** AI assistants can help users make decisions by providing information and recommendations.\n\nAI assistants are becoming increasingly popular as they become more sophisticated and capable. They are now used by millions of people around the world for a variety of tasks.\

## Connect with database and ask some basic question

In [3]:

from langchain.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_experimental.sql import SQLDatabaseChain

In [4]:
db_name=os.getenv("PG_DB")
user=os.getenv("PG_USERNAME")
password=os.getenv("PG_PASSWORD")
host=os.getenv("PG_HOST")
port=os.getenv("PG_PORT")

In [5]:
db = SQLDatabase.from_uri(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}")

c:\Users\Administrator\crm-venv310\lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(


In [6]:
print(db.table_info)


CREATE TABLE "AdTemplates" (
	id SERIAL NOT NULL, 
	"merchantId" INTEGER, 
	name VARCHAR(255), 
	leads INTEGER, 
	"createdAt" TIMESTAMP WITH TIME ZONE NOT NULL, 
	"updatedAt" TIMESTAMP WITH TIME ZONE NOT NULL, 
	"productId" INTEGER, 
	CONSTRAINT "AdTemplates_pkey" PRIMARY KEY (id), 
	CONSTRAINT "AdTemplates_merchantId_fkey" FOREIGN KEY("merchantId") REFERENCES "Merchants" (id) ON DELETE CASCADE ON UPDATE CASCADE, 
	CONSTRAINT fk_product_id FOREIGN KEY("productId") REFERENCES "Products" (id)
)

/*
3 rows from AdTemplates table:
id	merchantId	name	leads	createdAt	updatedAt	productId

*/


CREATE TABLE "Ads" (
	id SERIAL NOT NULL, 
	"merchantId" INTEGER, 
	"templateId" INTEGER, 
	read INTEGER, 
	delivered INTEGER, 
	sent INTEGER, 
	failed INTEGER, 
	response INTEGER, 
	"createdAt" TIMESTAMP WITH TIME ZONE NOT NULL, 
	"updatedAt" TIMESTAMP WITH TIME ZONE NOT NULL, 
	CONSTRAINT "Ads_pkey" PRIMARY KEY (id), 
	CONSTRAINT "Ads_merchantId_fkey" FOREIGN KEY("merchantId") REFERENCES "Merchants" 

In [17]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)

# create a query
response = chain.invoke({"question":   'How many categories do we have in my fake store' + ' Remember to add public."<table name>" for postgreql including the quotes on table name'})
response

'SELECT COUNT("name") FROM Categories WHERE "storeId"=10;'

Alternative to create an sql agent

In [18]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [19]:
cleaned_query = response.strip('```sql\n').strip('\n```')
cleaned_query

'SELECT COUNT("name") FROM Categories WHERE "storeId"=10;'

In [ ]:
db.run(cleaned_query)

In [13]:
from langchain.agents import create_sql_agent 
from langchain.agents import AgentExecutor 
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [14]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)

def execute_query(question):
    try:
        # Geerate SQL query from question
        response = chain.invoke({"question":   question + ' Remember to add public."<table name>" for postgreql including the quotes on table name'})
        print(response)
        print("########################")
        # clean the query
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print("########################")
        result = db.run(cleaned_query)
        print("########################")
        print(result)
    except Exception as e:
        print(f"An error occured", {e})
        return False

In [15]:
question = "List me the number of products sold for each brand in the store called My Trial Store"

execute_query(question)

```sql
SELECT
  b.name AS "Brand Name",
  COUNT(DISTINCT p.id) AS "Number of Products Sold"
FROM "Brands" AS b
JOIN "Products" AS p
  ON b.id = p.brandId
JOIN "Sales" AS s
  ON p.id = s.productId
JOIN "Stores" AS st
  ON s.storeId = st.id
WHERE
  st.name = 'My Trial Store'
GROUP BY
  b.name;
```
########################
SELECT
  b.name AS "Brand Name",
  COUNT(DISTINCT p.id) AS "Number of Products Sold"
FROM "Brands" AS b
JOIN "Products" AS p
  ON b.id = p.brandId
JOIN "Sales" AS s
  ON p.id = s.productId
JOIN "Stores" AS st
  ON s.storeId = st.id
WHERE
  st.name = 'My Trial Store'
GROUP BY
  b.name;
########################
An error occured {ProgrammingError('(psycopg2.errors.UndefinedColumn) column p.brandid does not exist\nLINE 6:   ON b.id = p.brandId\n                    ^\nHINT:  Perhaps you meant to reference the column "p.brandId".\n')}


False

In [16]:
agent_executor.run(question)



> Entering new SQL Agent Executor chain...


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8216\1032557354.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  agent_executor.run(question)


Action: sql_db_list_tables
Action Input: AdTemplates, Ads, Billboards, BotVectorStores, Brands, BulkTemplateTasks, ButtonRepliedActions, ButtonReplyActions, Categories, Chats, Conversations, Coupons, Customers, DocumentMessages, ImageHeaders, ImageMessages, Images, InteractiveButtons, InteractiveLists, InteractiveMessages, InteractiveTemplates, ListRepliedActions, ListRowButtons, ListSections, LocationMessages, Loyalties, MarketingResponses, Merchants, Messages, MpesaSettings, OrderItems, Orders, ProductCombinations, ProductImages, ProductVariants, ProductVariationOptions, ProductVariations, Products, PromotionCustomers, PromotionProducts, Promotions, Purchases, Redemptions, Rewards, SaleDetails, Sales, ScheduleTasks, SequelizeMeta, Settings, Stores, StripeSettings, TemplateRepliedActions, TemplateReplyActions, TextHeaders, TextMessages, VideoMessages, langchain_pg_collection, langchain_pg_embedding, migration, piccolo_user, sessionsThe Products table seems the most relevant.  I should

"I don't know"